# 보스턴 집값 예측하기
- scikit-learn 에서 정한 14개의 feature만 사용


## 데이터 불러오기

In [1]:
from sklearn import datasets
import pandas as pd

data_frame, y = datasets.fetch_openml('boston', return_X_y=True)

data_frame['target'] = y

# print(data_frame.head())
# print(len(data_frame.columns))

print(type(data_frame))

<class 'pandas.core.frame.DataFrame'>


/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:301: UserWarning: Multiple active versions of the dataset matching the name boston exist. Versions may be fundamentally different, returning version 1.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


## 학습 루프


## Batch and Epoch

- 데이터가 총 1000 개
- 만약 Batch Size : 100
> Batch 가 10 번이여야 모든 데이터 사용
- 1000 개의 데이터를 한 번 학습시키면 1 epoch
> epoch : Batch Size로 전체 데이터를 모두 학습하는 단위
- 이 때, iteration은 10
> iteration : Batch Size로 1 epoch 를 학습시킬 수 있는 배치 반복 횟수



In [2]:
import torch
import torch.nn as nn
import numpy as np

from torch.optim.adam import Adam

# 1. 모델 정의
model = nn.Sequential(
    nn.Linear(13, 100),
    nn.ReLU(),
    nn.Linear(100, 1)
)

# 2. 데이터 로드
X = data_frame.iloc[:, :13].values
Y = data_frame["target"].values

X=np.vstack(X).astype(np.float)
Y=np.vstack(Y).astype(np.float)


batch_size = 100
learning_rate = 0.001

# 3. Loss Function 정의
optim = Adam(model.parameters(), lr = learning_rate)

# epoch를 200으로 설정
for epoch in range(800):
  for i in range(len(X) // batch_size):
    start = i * batch_size
    end = start + batch_size

    x = torch.FloatTensor(X[start:end])
    y = torch.FloatTensor(Y[start:end])
    # x = torch.from_numpy(X[start:end])
    # y = torch.from_numpy(Y[start:end])

    # 가중치의 기울기 0으로 초기화
    optim.zero_grad()

    # 예측값 계산
    preds = model(x)

    # loss 계산
    loss = nn.MSELoss()(preds, y)

    # 오차 역전파
    loss.backward()

    # 최적화 진행
    optim.step()

  if epoch % 20 == 0:
    print(f"epoch {epoch} // loss : {loss}")


<ipython-input-2-9bfda1f97e3d>:18: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X=np.vstack(X).astype(np.float)
<ipython-input-2-9bfda1f97e3d>:19: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  Y=np.vstack(Y).astype(np.float)


epoch 0 // loss : 306.4560241699219
epoch 20 // loss : 16.20532989501953
epoch 40 // loss : 14.987722396850586
epoch 60 // loss : 13.490015983581543
epoch 80 // loss : 12.27328872680664
epoch 100 // loss : 11.75764274597168
epoch 120 // loss : 10.721931457519531
epoch 140 // loss : 10.519545555114746
epoch 160 // loss : 11.161391258239746
epoch 180 // loss : 10.449235916137695
epoch 200 // loss : 11.637478828430176
epoch 220 // loss : 10.646893501281738
epoch 240 // loss : 9.843896865844727
epoch 260 // loss : 20.358915328979492
epoch 280 // loss : 9.541389465332031
epoch 300 // loss : 12.615290641784668
epoch 320 // loss : 12.023781776428223
epoch 340 // loss : 10.344371795654297
epoch 360 // loss : 12.538782119750977
epoch 380 // loss : 11.200992584228516
epoch 400 // loss : 9.977457046508789
epoch 420 // loss : 10.46745491027832
epoch 440 // loss : 11.971872329711914
epoch 460 // loss : 12.153159141540527
epoch 480 // loss : 10.073965072631836
epoch 500 // loss : 8.73437213897705
ep

## 모델 성능 평가

In [3]:
prediction = model(torch.FloatTensor(X[0, :13]))
real = Y[0]

print(f"prediction : {prediction.item()} // real value : {real}")

prediction : 34.231204986572266 // real value : [24.]
